In [117]:
# works for python 3.9 
# !pip install s2fft
# !pip install s2wav
# !pip install jax


In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
#show graph inside the notebook


# Make sure we configure 64 bit precision. 
# 32 bit can be faster but you will be (potentially much) less precise.
import jax
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

import healpy as hp
import s2wav       # Wavelet transforms on the sphere and rotation group
import s2fft       # Spherical harmonic and Wigner transforms



In [85]:
PLA_Data = pd.read_csv("data/planck_simulation/PLA-Results.csv")

# Store the names of the datasets into PLA_Data_List
PLA_Data_List = PLA_Data['SIMULATED_MAP.FILE_ID'].to_list()


# Initialize PLA_Data_Dict : (key:frequnecy, value: [csv path])
PLA_Data_Dict = dict()
for each_csv_path in PLA_Data_List:
    PLA_Data_Dict[each_csv_path[20:23]] = ["data/planck_simulation/"+each_csv_path]
# display(PLA_Data_Dict)

# Read the CMB datasets and store them inside the PLA_Data_Dict
# Update PLA_Data_Dict : (key:frequnecy, value: [csv path, hp map dat ])
 
for frequency, storage_list in PLA_Data_Dict.items():
    storage_list.append(hp.read_map(storage_list[0], dtype=jnp.float64))
# display(PLA_Data_Dict)

# Convert the unit from unitMJy/steradian to K_CMB
# Source: https://wiki.cosmos.esa.int/planckpla2015/index.php/UC_CC_Tables

PLA_Data_Dict["545"][1] = PLA_Data_Dict["545"][1]/58.0356
PLA_Data_Dict["857"][1] = PLA_Data_Dict["857"][1]/2.2681
# PLA_Data_Dict["857"][1]

In [ ]:
# # Assuming PLA_Data_Dict contains your Planck map data
# Healpix_resolution_nside = hp.get_nside(PLA_Data_Dict["030"][1])
# f = PLA_Data_Dict["030"][1]

# L = 512  # Maximum multipole
# N = 4    # Number of wavelet directions

# # Load or generate wavelet filters
# filters = s2filters.from_name("spin{0}_{1}band".format(0, 3))

# # Perform wavelet analysis
# s2wav.analysis(f, L, N, sampling="healpix", nside=Healpix_resolution_nside, filters=filters)

In [123]:
L = 10              # Spherical harmonic bandlimit
N = 1                # Azimuthal (directional) bandlimit
nside = hp.get_nside(PLA_Data_Dict["030"][1])  # Resolution of HEALPix grid
# method = "jax_healpy"
sampling = "healpix"      # Sampling scheme
f = PLA_Data_Dict["030"][1]
f.shape

(12582912,)

In [125]:
filter_bank = s2wav.filters.filters_directional_vectorised(L, N)

# removed use_c_backend
# wavelet_coeffs, scaling_coeffs = s2wav.analysis(f, L, N, filters=filter_bank, use_c_backend=use_c_backend)
wavelet_coeffs, scaling_coeffs = s2wav.analysis(f, L, N, sampling = sampling, nside = nside, filters= filter_bank)

TypeError: iota shape must have every element be nonnegative, got (-2,).

In [116]:
help(s2wav.analysis_base)
help(s2wav.analysis)

Help on function analysis in module s2wav.transforms.base:

analysis(f: numpy.ndarray, L: int, N: int = 1, J_min: int = 0, lam: float = 2.0, spin: int = 0, spin0: int = 0, sampling: str = 'mw', nside: int = None, reality: bool = False, multiresolution: bool = True, scattering: bool = False) -> Tuple[numpy.ndarray, numpy.ndarray]
    Wavelet analysis from pixel space to wavelet space for complex signals.
    
    Args:
        f (np.ndarray): Signal :math:`f` on the sphere with shape :math:`[n_{\theta}, n_{\phi}]`.
        L (int): Harmonic bandlimit.
        N (int, optional): Upper azimuthal band-limit. Defaults to 1.
        J_min (int, optional): Lowest frequency wavelet scale to be used. Defaults to 0.
        lam (float, optional): Wavelet parameter which determines the scale factor between consecutive wavelet scales.
            Note that :math:`\lambda = 2` indicates dyadic wavelets. Defaults to 2.
        spin (int, optional): Spin (integer) of input signal. Defaults to 0.
    

In [66]:
# f_check = s2wav.synthesis_base(wavelet_coeffs, scaling_coeffs, L, N, filters=filter_bank)
# f_check = s2wav.synthesis_base(wavelet_coeffs, scaling_coeffs, L, N, filters=filter_bank)



In [67]:
print(f"Mean absolute error = {np.nanmean(np.abs(f_check - f))}")

Mean absolute error = 2.1150765748284385e-14


In [105]:
# Get Healpix resolution (nside)
Healpix_resolution_nside = hp.get_nside(PLA_Data_Dict["030"][1])


f = PLA_Data_Dict["030"][1]

L = 512  # Maximum multipole
N = 4    # Number of wavelet directions

filter_bank = s2wav.filters.filters_directional_vectorised(L, N)
wavelet_coeffs, scaling_coeffs = s2wav.analysis(f, L, N, sampling = "healpix", filters = filter_bank, nside = Healpix_resolution_nside)
# help(s2wav.analysis)





TypeError: iota shape must have every element be nonnegative, got (-2,).

In [ ]:
s2wav.synthesis(f_wav, f_scal, L, N)